In [ ]:
from load_data_and_augmentations import *

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
plt.rcParams['figure.figsize'] = (14,7)
font = {'family' : 'DejaVu Sans',  'weight' : 'normal',  'size'  : 20}
plt.rc('font', **font)

In [ ]:
tensor_transform = get_tensor_transform('Kinetics', True)
train_spat_transform = get_spatial_transform(2)
train_temp_transform = get_temporal_transform(30)
valid_spat_transform = get_spatial_transform(0)
valid_temp_transform = va.TemporalFit(size=30)

root_dir = '/media/scratch/astamoulakatos/centre_Ch2/'
df = pd.read_csv('../important_csvs/more_balanced_dataset/more_balanced_stratified.csv')


In [ ]:
#df = df.sample(frac=0.03)

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df

In [ ]:
#df = df[df.number_of_frames>=100]

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df.label.value_counts()

In [ ]:
df_train.label.value_counts()

In [ ]:
df_train = get_df(df, 50, True, False, False)
class_image_paths, end_idx, idx_label= get_indices(df_train, root_dir)
#train_loader = get_loader(16, 64, end_idx, class_image_paths, train_temp_transform, train_spat_transform, tensor_transform, False, False)

In [ ]:
class_image_paths

In [ ]:
idx_label

In [ ]:
len(df_train)

In [ ]:
len(end_idx)

In [ ]:
len(idx_label)

In [ ]:
len(class_image_paths), len(end_idx)

In [ ]:
idx_label.count('exp'), idx_label.count('exp_and')

In [ ]:
end_idx

In [ ]:
seq_length_list = []
for i in range(len(end_idx)-1):
    seq_length_list.append((end_idx[i+1]-end_idx[i]).item())

In [ ]:
len(seq_length_list)

In [ ]:
seq_lengths = torch.tensor(seq_length_list)

In [ ]:
seq_lengths, end_idx

# my new sampler

In [ ]:
import random

In [ ]:
class My_new_Sampler(torch.utils.data.Sampler):
    def __init__(self, end_idx):
        end_idx = end_idx[:-1]
        indices = end_idx
        seq_length_list = []
        for i in range(len(end_idx)-1):
            seq_length_list.append((end_idx[i+1]-end_idx[i]).item())
        seq_lengths = torch.tensor(seq_length_list)        
        self.indices = indices
        self.seq_lengths = seq_lengths
        
    def __iter__(self):
        torch.manual_seed(0)
        indices = self.indices[torch.randperm(len(self.indices))]
        seq_lengths = self.seq_lengths[torch.randperm(len(self.seq_lengths))]
        indices = indices.tolist()
        seq_lengths = seq_lengths.tolist()
        index_plus_seq = zip(indices, seq_lengths)
        #random.shuffle(list(index_plus_seq))
        return iter(index_plus_seq)
    
    def __len__(self):
        return len(self.indices)

In [ ]:
sampler = My_new_Sampler(end_idx)

In [ ]:
sampler.indices

In [ ]:
sampler.seq_lengths

In [ ]:
len(sampler)

In [ ]:
class MyDataset_different_seq(Dataset):
    def __init__(self, image_paths, temp_transform, spat_transform, tensor_transform, length, lstm=False, oned = False, augment = False, multi = 1): #csv_file, 
        self.image_paths = image_paths
        #self.seq_length = seq_length
        self.temp_transform = temp_transform
        self.spat_transform = spat_transform
        self.tensor_transform = tensor_transform
        self.length = length
        self.lstm = lstm
        self.oned = oned
        self.augment = augment
        self.multi = multi
        
    def __getitem__(self, index):
        start = index[0]
        end = index[0] + index[1]
        print('Getting images from {} to {}'.format(start, end))
        indices = list(range(start, end))
        images = []
        #tr = self.transform
        for i in indices:
            image_path = self.image_paths[i][0]
            image = Image.open(image_path)
            images.append(image)
        x = images
        if not self.oned:
            x = self.temp_transform(x)
        if self.augment:
            x = self.spat_transform(x)
        x = self.tensor_transform(x)
        y = torch.tensor([self.image_paths[start][self.multi]], dtype=torch.long)
        y = y.squeeze(dim=0)
        y = y.float()
        #print(y.shape)
        if self.lstm:
            x = x.permute(1,0,2,3)
        if self.oned:
            x = x.squeeze(dim=1)
        return x, y
    
    def __len__(self):
        return self.length

In [ ]:
dataset = MyDataset_different_seq(
        image_paths = class_image_paths,
        temp_transform = valid_temp_transform,
        spat_transform = valid_spat_transform,
        tensor_transform = tensor_transform,
        length = len(sampler),
        lstm = False,
        oned = False,
        augment = True,
        multi = 1)

In [ ]:
train_loader = DataLoader(
        dataset,
        batch_size = 100,
        sampler = sampler,
        drop_last = True,
        num_workers = 0)

In [ ]:
len(idx_label)

In [ ]:
show_batch(train_loader, 50, True)

# Random Shuffle of both seq_lengths and indices in the same order

In [ ]:
torch.manual_seed(0)

In [ ]:
end_idx = end_idx[torch.randperm(len(end_idx))]
seq_lengths = seq_lengths[torch.randperm(len(seq_lengths))]

In [ ]:
end_idx

In [ ]:
seq_lengths

In [ ]:
seq_length = 50

In [ ]:
from itertools import repeat

In [ ]:
indices = []
label_indices = []
for i in range(len(end_idx) - 1):
    start = end_idx[i] #+ 40 #so that we dont use the first 40 frames, especially in 2d resnet!
    start_label = idx_label[i]
    end = end_idx[i + 1] - seq_length #- 20 #so that we dont use the last 20 frames
    if start > end:
        pass
    else:
        label_indices.extend(repeat(start_label,end-start))
        indices.append(torch.arange(start, end))
indices = torch.cat(indices)
#self.indices = indices

In [ ]:
len(indices)

In [ ]:
len(label_indices)

In [ ]:
label_indices.count('exp'), label_indices.count('exp_and'), label_indices.count('bur'), label_indices.count('exp_fj'), label_indices.count('exp_fs')

In [ ]:
df_label_indices = pd.DataFrame({'label_indices':label_indices})
df_label_indices

In [ ]:
df_train.label.value_counts()

# Indices Distribution

In [ ]:
df_label_indices.label_indices.value_counts().plot(kind='barh')

In [ ]:
indices[0:200]

In [ ]:
label_indices[0:200]

In [ ]:
indices = indices[torch.randperm(len(indices))]
t = iter(indices.tolist())

In [ ]:
indices

In [ ]:
len(indices)

In [ ]:
from imbalanced_sampler_3 import MultilabelBalancedRandomSampler

In [ ]:
labels = []
for i in class_image_paths:
    labels.append(i[2])

In [ ]:
def unique(list1): 
  
    # intilize a null list 
    unique_list = [] 
      
    # traverse for all elements 
    for x in list1: 
        # check if exists in unique_list or not 
        if x not in unique_list: 
            unique_list.append(x) 
    # print list 
    for x in unique_list: 
        print(x)

In [ ]:
len(labels)

In [ ]:
len(indices)

In [ ]:
unique(labels)

In [ ]:
labels = np.array(labels)

In [ ]:
train_sampler = MultilabelBalancedRandomSampler(
    labels, indices, class_choice="least_sampled"
)

In [ ]:
dataset = MyDataset(
        image_paths = class_image_paths,
        seq_length = 100,
        temp_transform = valid_temp_transform,
        spat_transform = valid_spat_transform,
        tensor_transform = tensor_transform,
        length = len(train_sampler),
        lstm = False,
        oned = False,
        augment = True,
        multi = 1)

In [ ]:
len(dataset)

In [ ]:
train_loader = DataLoader(
        dataset,
        batch_size = 10,
        sampler = train_sampler,
        drop_last = True,
        num_workers = 0)

In [ ]:
df_valid = get_df(df, 100, False, True, False)
class_image_paths, end_idx = get_indices(df_valid, root_dir)
valid_loader = get_loader(100, 10, end_idx, class_image_paths, valid_temp_transform, valid_spat_transform, tensor_transform, False, False, False, 1)

In [ ]:
df_train.label.value_counts()

In [ ]:
sum_ = torch.zeros(5); sum_

In [ ]:
print("================ Training phase ===============")
sum_ = torch.zeros(5)
for x, y in train_loader:
    print("Label counts per class:")
    sum_ = sum_ + y.sum(axis=0)
    #sum_ = y.sum(axis=0)
    print(sum_) 
    print("Difference between min and max")
    print(max(sum_) - min(sum_))
    print("")
print("")


In [ ]:
sum_

In [ ]:
show_batch(train_loader, 10, True)

In [ ]:
dataset[0][0].shape

In [ ]:
dataset[0][1].shape

In [ ]:
print("=============== Validation phase ==============")
for x, y in valid_loader:
    print("Label counts per class:")
    sum_ = y.sum(axis=0)
    print(sum_)
    print("Difference between min and max")
    print(max(sum_) - min(sum_))
    print("")
print("")


In [ ]:
show_batch(valid_loader, 10 , False)

In [ ]:
inputs, classes = next(iter(train_loader))

In [ ]:
inputs.shape[1]